# Data loading and data cleaning (Madrid accident data)

In [33]:
import pandas as pd
import numpy as np
import altair as alt
import os.path
import urllib.request

## Case description

In this case study we access public data and perform an exploratory analysis and data cleaning and preparation.
The data to be analyzed correspond to traffic accidents in the city of Madrid, which are available through the open data portal of the Madrid City Council. It is a dataset with multiple dimensions of different types and a small cardinality, so it is simple to manipulate but rich.

## Descarga de los datos

First we show how to download the data and store them locally. This step does not need to be repeated each time this code is run, so we first check if the files already exist so as not to download them again.

(In this example, this means of accessing the data is commented out, and is replaced by a local access, so that it is possible to run the notebook even without an internet connection or even if the site datos.madrid.es is no longer available. However, it is shown here for didactic purposes).

In [2]:
# If the 'data' directory does not exist, we create it

# if not os.path.exists('./datos'):
#     os.makedirs('./datos')

# # URLs of the data to be accessed and name of the files in which to save them
# URL_DATOS_ACCIDENTES = 'https://datos.madrid.es/egob/catalogo/300228-19-accidentes-trafico-detalle.csv'
# PATH_DATOS_ACCIDENTES = './datos/accidentes2019.csv'

# # The municipality offers a small document with the description of the data, 
# # which we take the opportunity to download as well.
# # (From Jupyter Lab, just double click on the file name in the left panel 
# # so that it opens in a new tab)
# URL_PDF_ACCIDENTES = 
# 'https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_DS_Accidentes_trafico_desde_2019.pdf'
# PATH_PDF_ACCIDENTES = './datos/accidentes2019.pdf'

# if not os.path.exists(PATH_DATOS_ACCIDENTES):
#     urllib.request.urlretrieve(URL_DATOS_ACCIDENTES, PATH_DATOS_ACCIDENTES)
    
# if not os.path.exists(PATH_PDF_ACCIDENTES):
#     urllib.request.urlretrieve(URL_PDF_ACCIDENTES, PATH_PDF_ACCIDENTES)

## Data reading

Once the data is downloaded, which in this case is in csv format, the easiest thing to do is to load it in a Pandas DataFrame to proceed with an initial exploration. In this particular case, for this load to work correctly it is necessary to specify the encoding of the csv file, since it does not use the usual UTF-8 standard but contains special characters (“special” for the ASCII standard, such as the letter eñe or accented vowels).

First, we look at the first lines of the file, to get an idea of its contents

In [35]:
with open('./datos/accidentes2019.csv', encoding = 'latin1') as f:
    for _ in range(5) : print(f.readline())

Nº  EXPEDIENTE;FECHA;HORA;CALLE;NÚMERO;DISTRITO;TIPO ACCIDENTE;ESTADO METEREOLÓGICO;TIPO VEHÍCULO;TIPO PERSONA;RANGO EDAD;SEXO;LESIVIDAD*;* La correspondencia de los códigos se encuentra descrito en la estructura del fichero.

2019S000020;01/01/2019;23:30;CALL. FUENCARRAL;149;CHAMBERÍ;Caída;Despejado;Ciclomotor;Conductor;DE 25 A 29 AÑOS;Hombre;01;

2019S000017;01/01/2019;22:15;CALL. OCA / CALL. PINZON;-;CARABANCHEL;Colisión fronto-lateral;Despejado;Turismo;Conductor;DE 40 A 44 AÑOS;Mujer;14;

2019S000017;01/01/2019;22:15;CALL. OCA / CALL. PINZON;-;CARABANCHEL;Colisión fronto-lateral;Despejado;Ciclomotor;Conductor;DE 35 A 39 AÑOS;Hombre;03;

2019S001812;01/01/2019;21:40;CALL. BAILEN / CUSTA. SAN VICENTE;-;CENTRO;Colisión fronto-lateral;Despejado;Turismo;Conductor;DE 40 A 44 AÑOS;Hombre;14;



From this initial inspection we can draw some conclusions (which could also be obtained by trial and error with pd.read_csv():
- The field separator is the ; character (this is usual in csv files in Spain, where the comma is used to separate the decimal part of the numbers).
- The first line of the file contains the field names, as usual in many csv files.
- The second field, “DATE”, is a date type data, and the third, “TIME” is a time of day.
- There is a comment at the end of the first row, which would be interpreted as the name of one more column of data, which would need to be deleted.

Next, we read the raw data into a Pandas dataframe, so that we can process it more comfortably, and we show the first five rows of data and a general description of the dataframe contents

In [36]:
accidentes_raw = pd.read_csv('./datos/accidentes2019.csv', encoding = 'latin1', sep = ';')
accidentes_raw.head(5)

,Nº EXPEDIENTE,FECHA,HORA,CALLE,NÚMERO,DISTRITO,TIPO ACCIDENTE,ESTADO METEREOLÓGICO,TIPO VEHÍCULO,TIPO PERSONA,RANGO EDAD,SEXO,LESIVIDAD*,* La correspondencia de los códigos se encuentra descrito en la estructura del fichero.
0,2019S000020,01/01/2019,23:30,CALL. FUENCARRAL,149,CHAMBERÍ,Caída,Despejado,Ciclomotor,Conductor,DE 25 A 29 AÑOS,Hombre,1.0,NaN
1,2019S000017,01/01/2019,22:15,CALL. OCA / CALL. PINZON,-,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 40 A 44 AÑOS,Mujer,14.0,NaN
2,2019S000017,01/01/2019,22:15,CALL. OCA / CALL. PINZON,-,CARABANCHEL,Colisión fronto-lateral,Despejado,Ciclomotor,Conductor,DE 35 A 39 AÑOS,Hombre,3.0,NaN
3,2019S001812,01/01/2019,21:40,CALL. BAILEN / CUSTA. SAN VICENTE,-,CENTRO,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 40 A 44 AÑOS,Hombre,14.0,NaN
4,2019S001812,01/01/2019,21:40,CALL. BAILEN / CUSTA. SAN VICENTE,-,CENTRO,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 30 A 34 AÑOS,Mujer,7.0,NaN


In [37]:
accidentes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51806 entries, 0 to 51805
Data columns (total 14 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   Nº  EXPEDIENTE                                                                           51806 non-null  object 
 1   FECHA                                                                                    51806 non-null  object 
 2   HORA                                                                                     51806 non-null  object 
 3   CALLE                                                                                    51806 non-null  object 
 4   NÚMERO                                                                                   51802 non-null  object 
 5   DISTRITO                                                    

Indeed, we see that a last column has appeared with all the NaN (“Not a Number”) values that we can ignore.

## Data cleaning

We are going to analyze and clean column by column, since we see that each one is a different case. To make it easier, we write a small function that provides us with the most relevant data for this cleanup

In [38]:
def describe_columna(df, col):
    print(f'Columna: {col}  -  Tipo de datos: {df[col].dtype}')
    print(f'Número de valores nulos: {df[col].isnull().sum()}  -  Número de valores distintos: {df[col].nunique()}')
    print('Valores más frecuentes:')
    for i, v in df[col].value_counts().iloc[:10].items() :
        print(i, '\t', v)

In [39]:
describe_columna(accidentes_raw, 'Nº  EXPEDIENTE')

Columna: Nº  EXPEDIENTE  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 21935
Valores más frecuentes:
2019S036555 	 13
2019S032214 	 12
2019S038336 	 12
2019S029891 	 12
2019S000625 	 12
2019S028162 	 12
2019S013324 	 12
2019S017281 	 11
2019S022408 	 11
2019S024660 	 11


Nothing particularly interesting in this column. We see that the file number is not unique (the most used value, the first in the list of values, is used a total of 13 times): reading the data description document, or simply inspecting the data, we can see that the file number identifies an accident, but it may have involved several vehicles or more than one person, and for each of them there is a line in this file. The number of accidents, therefore, is not the number of rows in the file, but the number of unique values in the Dossier 'Nº EXPEDIENTE' field.

For the field-by-field cleanup, we will create a new dataframe to which we will assign each column already processed. In this case, the only change we make is to correct the field name so that it does not have a double space between 'Nº' and 'EXPEDIENTE', and to change the name to lowercase.

In [40]:
accidentes = pd.DataFrame()
accidentes['Nº Expediente'] = accidentes_raw['Nº  EXPEDIENTE']

Next column...

In [41]:
describe_columna(accidentes_raw, 'FECHA')
print()
describe_columna(accidentes_raw, 'HORA')

Columna: FECHA  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 365
Valores más frecuentes:
05/04/2019 	 272
31/05/2019 	 239
24/04/2019 	 234
20/12/2019 	 229
01/02/2019 	 229
12/12/2019 	 226
22/02/2019 	 212
19/01/2019 	 210
04/10/2019 	 209
31/10/2019 	 207

Columna: HORA  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 1213
Valores más frecuentes:
18:00 	 464
19:30 	 457
16:00 	 452
20:00 	 452
18:30 	 441
17:00 	 424
13:00 	 421
21:00 	 405
14:30 	 395
14:00 	 395


We see that both dates and times have been read as data type “object”, which is the way Pandas identifies character strings (at least, until version 1.0, from which there is a specific type, string, for character strings). To make it easier to operate with them, we transform them to data type “datetime” and put them together in a single field that includes both date and time (to be able to sort the accidents, easily search all accidents between any two concrete instants, etc.).

In [42]:
accidentes['Fecha'] = pd.to_datetime(accidentes_raw.FECHA) + \
                      pd.to_timedelta(accidentes_raw.HORA + ':00')

Next columns...

In [43]:
describe_columna(accidentes_raw, 'CALLE')

Columna: CALLE  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 10456
Valores más frecuentes:
PASEO. CASTELLANA 	 719
CALL. ALCALA 	 646
CALL. BRAVO MURILLO 	 265
AVDA. ALBUFERA 	 264
CALL. FRANCISCO SILVELA 	 179
CALL. SERRANO 	 177
PASEO. SANTA MARIA DE LA CABEZA 	 169
PASEO. EXTREMADURA 	 167
CALL. PRINCIPE DE VERGARA 	 167
CALL. DOCTOR ESQUERDO 	 153


In [44]:
accidentes['Calle'] = accidentes_raw['CALLE']

In [45]:
describe_columna(accidentes_raw, 'NÚMERO')

Columna: NÚMERO  -  Tipo de datos: object
Número de valores nulos: 4  -  Número de valores distintos: 604
Valores más frecuentes:
- 	 28523
1 	 1489
2 	 1077
3 	 701
4 	 642
0 	 519
7 	 494
5 	 448
8 	 421
6 	 388


It might seem like a good idea to convert the “NUMBER” field to a numeric type, but let's first check how many values could be converted...

In [46]:
# innumeric() tells us whether a value can be transformed into a numeric type or not.
accidentes_raw['NÚMERO'].str.isnumeric().value_counts()

False    29306
True     22496
Name: NÚMERO, dtype: int64

More than half of the values cannot be converted to numbers. Let's check what these fields contain

In [47]:
# we select the accident_raw records for which isnumeric() of the “NUMBER” field is false,
# we keep that field, and count the most repeated values.
accidentes_raw[accidentes_raw['NÚMERO'].str.isnumeric() == False]['NÚMERO'].value_counts()

-        28523
 1A         68
 5A         35
 2A         30
 259A       19
         ...  
 45A         1
 7D          1
 89A         1
2O           1
 334A        1
Name: NÚMERO, Length: 161, dtype: int64

We see that, although the field is called “NUMBER”, it contains values that are text strings. We also see there that there is a very frequent value, the “-” character, which is used to indicate that no information is available (it has no additional meaning). It is always convenient to normalize the values corresponding to the missing data, and in this case, the best way to do it is to use the standard Pandas mechanism to mark missing data, the NaN values

In [48]:
accidentes['Número'] = accidentes_raw['NÚMERO']
accidentes.loc[accidentes['Número'] == '-', 'Número'] = np.NaN

Next field...

In [49]:
describe_columna(accidentes_raw, 'DISTRITO')

Columna: DISTRITO  -  Tipo de datos: object
Número de valores nulos: 5  -  Número de valores distintos: 21
Valores más frecuentes:
SALAMANCA 	 4078
PUENTE DE VALLECAS 	 4063
CHAMARTÍN 	 3891
CARABANCHEL 	 3365
CIUDAD LINEAL 	 3362
SAN BLAS-CANILLEJAS 	 2797
CENTRO 	 2752
MONCLOA-ARAVACA 	 2688
CHAMBERÍ 	 2680
RETIRO 	 2671


As the number of unique values of this field is small, and also the possible values are limited (there are no more districts in Madrid than these, any other value should not be accepted), it is convenient to convert the value to “category” type, as this reduces the storage needed, speeds up the calculations and facilitates the analysis.

In [50]:
accidentes['Distrito'] = accidentes_raw['DISTRITO'].astype('category')

The following five fields present similar cases to the case of “DISTRITO”, and are treated in the same way

In [51]:
for col in accidentes_raw.columns[7:12].to_list():
    describe_columna(accidentes_raw, col)
    print()

Columna: ESTADO METEREOLÓGICO  -  Tipo de datos: object
Número de valores nulos: 5132  -  Número de valores distintos: 7
Valores más frecuentes:
Despejado 	 39984
Lluvia débil 	 3465
Nublado 	 1862
Se desconoce 	 695
LLuvia intensa 	 652
Granizando 	 14
Nevando 	 2

Columna: TIPO VEHÍCULO  -  Tipo de datos: object
Número de valores nulos: 178  -  Número de valores distintos: 29
Valores más frecuentes:
Turismo 	 36499
Motocicleta > 125cc 	 3527
Furgoneta 	 3125
Motocicleta hasta 125cc 	 2529
Autobús 	 1408
Camión rígido 	 1167
Bicicleta 	 884
Ciclomotor 	 809
Todo terreno 	 689
Otros vehículos con motor 	 330

Columna: TIPO PERSONA  -  Tipo de datos: object
Número de valores nulos: 25  -  Número de valores distintos: 3
Valores más frecuentes:
Conductor 	 41604
Pasajero 	 8357
Peatón 	 1820

Columna: RANGO EDAD  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 18
Valores más frecuentes:
DE 40 A 44 AÑOS 	 5736
DE 25 A 29 AÑOS 	 5610
DE 35 A 39 AÑOS 	 55

In [52]:
for col in accidentes_raw.columns[7:12].to_list():
    accidentes[col.capitalize()] = accidentes_raw[col].astype('category')

accidentes.loc[accidentes['Estado metereológico'] == 'Se desconoce', 'Estado metereológico'] = np.NaN

accidentes.loc[accidentes['Rango edad'] == 'DESCONOCIDA', 'Rango edad'] = np.NaN

A detail about the “Rango edad” field: when the values of a categorical field have a natural order, as in this case, it is convenient to represent them by means of an ordered categorical data type, because this facilitates the analysis, allows to make intervals, makes the graphs come out automatically ordered, etc. In this case, the automatic order assigned to the categories, which is alphabetical, does not coincide with the real order, so it has to be assigned manually.

In [53]:
rangos_edad = pd.CategoricalDtype(
    ['DE 0 A 5 AÑOS', 'DE 6 A 9 AÑOS', 'DE 10 A 14 AÑOS', 'DE 15 A 17 AÑOS',
     'DE 18 A 20 AÑOS', 'DE 21 A 24 AÑOS', 'DE 25 A 29 AÑOS', 'DE 30 A 34 AÑOS', 
     'DE 35 A 39 AÑOS', 'DE 40 A 44 AÑOS', 'DE 45 A 49 AÑOS', 'DE 50 A 54 AÑOS',
     'DE 55 A 59 AÑOS', 'DE 60 A 64 AÑOS', 'DE 65 A 69 AÑOS', 'DE 70 A 74 AÑOS', 
     'MAYOR DE 74 AÑOS'],
    ordered=True)

accidentes['Rango edad'] = accidentes['Rango edad'].astype(rangos_edad)

Last column...

In [54]:
describe_columna(accidentes_raw, 'LESIVIDAD*')

Columna: LESIVIDAD*  -  Tipo de datos: float64
Número de valores nulos: 21776  -  Número de valores distintos: 9
Valores más frecuentes:
14.0 	 16599
7.0 	 7110
2.0 	 2157
6.0 	 1555
1.0 	 1320
5.0 	 715
3.0 	 539
4.0 	 34
77.0 	 1


The “LESIVIDAD” (lesivity) field presents an interesting case: the values are coded, and the key to understanding them is in the data explanation document. There the values are described in this way:

Lesivity | Description
--- | --- 
01        |  Emergency care without subsequent admission - LEVE
02        |  Admission less than or equal to 24 hours - LEVE 
03        |  Admission more than 24 hours - GRAVE
04        |  Deceased 24 hours - FALLECIDO
05        |  Subsequent outpatient health care - LEVE
06        |  Immediate health care at a health center or mutual insurance company - LEVE
07        |  Health care only at the accident site - LEVE
14        |  No health care
77        |  Unkonwn
En blanco |  No health care

To normalize, we can first assign the blank values (read with NaN in accident_raw) to category 14, and the values of category 77 can be assigned as NaN, which is the way to represent missing or unknown data. Additionally, in order to have accessible the injury severity information, which is coded in a hidden way in the different injury categories, we can create an additional field with those values, which can also be a sorted category with the advantages that this has.

In [55]:
accidentes['Lesividad'] = accidentes_raw['LESIVIDAD*']

In [56]:
accidentes.loc[accidentes['Lesividad'].isnull(), 'Lesividad']  = 14
accidentes.loc[accidentes['Lesividad'] == 77, 'Lesividad'] = np.NaN

In [57]:
# Definition of the categorical type for degrees of severity
grados_gravedad = pd.CategoricalDtype(['Ileso', 'Leve', 'Grave', 'Fallecido'], ordered=True)

# We define a dictionary that maps the values of Lesivity with those of Severity.
# For convenience, we do not include the values corresponding to 'Mild', but use that 
# is used as the default value
dict_gravedad = {14: 'Ileso', 3: 'Grave', 4: 'Fallecido'} 

# We generate a new field 'Gavedad' by calculating the value corresponding to the degree of Lesividad
# of each case, and converting it to the ordered categorical type 'degrees_gravity'.
accidentes['Gravedad'] = accidentes['Lesividad'].apply(
    lambda x: dict_gravedad.get(x,'Leve')).astype(grados_gravedad)

## Testing and analysis

Once the data has been cleaned, we can check the generated dataframe...

In [58]:
accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51806 entries, 0 to 51805
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Nº Expediente         51806 non-null  object        
 1   Fecha                 51806 non-null  datetime64[ns]
 2   Calle                 51806 non-null  object        
 3   Número                23279 non-null  object        
 4   Distrito              51801 non-null  category      
 5   Estado metereológico  45979 non-null  category      
 6   Tipo vehículo         51628 non-null  category      
 7   Tipo persona          51781 non-null  category      
 8   Rango edad            46473 non-null  category      
 9   Sexo                  46756 non-null  category      
 10  Lesividad             51805 non-null  float64       
 11  Gravedad              51806 non-null  category      
dtypes: category(7), datetime64[ns](1), float64(1), object(3)
memory usage: 2.3

... and proceed to exploratory analysis (e.g., by performing summations or making graphs to better understand the data).

In [59]:
top10_vehiculos = accidentes['Tipo vehículo'].value_counts()[:10].index.tolist()

In [60]:
# Total number of accident victims by type of vehicle and role of the accident victim.
# We group the records by the fields “Type of vehicle” and “Type of person”, 
# select the field “File No.”, count the number of records in each group, 
# put the data in the form of a cross table and filter rows,
# count the number of records in each group, put the data in the form of a cross table and filter the rows
# to select only those that correspond to the 10 most frequent vehicle types.

accidentes.groupby(['Tipo vehículo', 'Tipo persona'])['Nº Expediente'].count().unstack().loc[top10_vehiculos]

Tipo persona,Conductor,Pasajero,Peatón
Tipo vehículo,,,
Turismo,28523,6734,1224
Motocicleta > 125cc,3179,232,115
Furgoneta,2616,371,135
Motocicleta hasta 125cc,2282,166,80
Autobús,893,467,47
Camión rígido,1040,98,29
Bicicleta,800,5,79
Ciclomotor,726,64,18
Todo terreno,539,120,30


In [61]:
# Percentage of accident victims by type of vehicle and sex
# First we make a selection similar to the one in the previous point, and then, using the function apply,
# we calculate the division of each value by the sum of values in that row, and display it formatted as a percentage.

acc_por_sexo = accidentes.groupby(['Tipo vehículo', 'Sexo'])['Nº Expediente'].count().unstack().loc[top10_vehiculos]

acc_por_sexo.apply(lambda x: x/x.sum(), axis=1).style.format("{:.1%}")

Sexo,Hombre,Mujer
Tipo vehículo,,
Turismo,61.5%,38.5%
Motocicleta > 125cc,84.0%,16.0%
Furgoneta,85.7%,14.3%
Motocicleta hasta 125cc,77.0%,23.0%
Autobús,69.1%,30.9%
Camión rígido,94.6%,5.4%
Bicicleta,74.3%,25.7%
Ciclomotor,75.0%,25.0%
Todo terreno,66.6%,33.4%


In [62]:
# Accident victims by gender and age range
# We generate the data as follows: we group by age range and sex, select the field “File No.”, 
# count the number of records and reset the index,
# we count the number of records and reset the index 
# (the latter, so that the “Age Range” and “Sex” data appear as columns of the dataset, not as levels of the index)
# We then display the data in a bar chart trellis.

data = accidentes.groupby(['Rango edad', 'Sexo'])['Nº Expediente'].count().reset_index()

alt.Chart(data, width=300, height=150, title='Accidentes en Madrid durante 2019').mark_bar().encode(
    x = alt.X('Rango edad:O', sort = rangos_edad.categories.to_list(), title = None),
    y = alt.Y('Nº Expediente:Q', title = 'Numero de implicados'),
    facet=alt.Facet('Sexo:N', title = None))

alt.Chart(...)

In [63]:
# Accidents by district
# We generate the data as follows: we group by district, select the field “File No.”,
# count the number of unique records (to count accidents, not persons injured) and reset the index. 
# We then display the data in a simple bar chart (ordering the X coordinate, i.e. the districts,
# in reverse order of the value of the Y-coordinate, i.e. the number of accidents)

data = accidentes.groupby(['Distrito'])['Nº Expediente'].nunique().reset_index()

alt.Chart(data, width=350, height=200, title='Accidentes en Madrid durante 2019').mark_bar().encode(
    x = alt.X('Distrito:N', sort = '-y', title = None),
    y = alt.Y('sum(Nº Expediente):Q', title = 'Numero de accidentes'))

alt.Chart(...)